<a href="https://colab.research.google.com/github/anuradhasurabhi/RNN/blob/master/Imagecaptioning_Flickr8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.layers import Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import numpy as np


In [ ]:
prev_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))
#print(prev_model.summary())
new_model = Sequential()
new_model.add(prev_model)
new_model.add(Flatten())
new_model.add(Dense(1024,activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(1024,activation='relu'))

new_model.summary()

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(optimizer=sgd, loss='categorical_crossentropy')

58892288/58889256 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              1049600   
Total params: 41,455,424
Trainable params: 41,455,424
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from os import listdir
from pickle import dump
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model

# extract features from each photo in the directory
def extract_features(directory):
  features = dict()
  for name in listdir(directory):
    filename = directory + '/' + name
    image = load_img(filename, target_size=(224, 224))
    #print(image)
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    #print(image)
    image = preprocess_input(image)
    feature = new_model.predict(image, verbose=0)
    image_id = name.split('.')[0]
    features[image_id] = feature
  return features

directory = images_path
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

Extracted Features: 40


In [ ]:
#TESTING
directory = images_path                          # assign images directory
features = extract_features(directory)           # extract features from all images
print('Extracted Features: %d' % len(features))  # print No. of features extracted
print(features['123997871_6a9ca987b1'])          # print feature vector of img_id = '123997871_6a9ca987b1'
print(features['141755292_7a0b3364cf'])          # print feature vector of img_id = '141755292_7a0b3364cf'
print(features['141755292_7a0b3364cf'].shape)    # display shape of each feature 

Extracted Features: 40
[[ 4.4933434  0.         4.6346717 ... 14.779005   8.231546   0.       ]]
[[ 0.        0.       11.115056 ...  0.       12.766032  9.64352 ]]
(1, 1024)


ORIGINAL CODE BEGINS HERE

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from numpy import array
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

In [3]:
import os
os.listdir('drive/My Drive/Flickr_Data')

['Images',
 'Flickr_TextData',
 'flickr8ktextfiles',
 'test_example1.jpg',
 'test_example2.jpg',
 'test_example3.jpg',
 'features.pkl']

In [4]:
images_dir = os.listdir("drive/My Drive/Flickr_Data")

images_path = 'drive/My Drive/Flickr_Data/Images'
captions_path = 'drive/My Drive/Flickr_Data/Flickr_TextData/Flickr8k.token.txt'
train_path = 'drive/My Drive/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt'
val_path = 'drive/My Drive/Flickr_Data/Flickr_TextData/Flickr_8k.devImages.txt'
test_path = 'drive/My Drive/Flickr_Data/Flickr_TextData/Flickr_8k.testImages.txt'
new_img1 = 'drive/My Drive/Flickr_Data/test_example1.jpg'
new_img2 = 'drive/My Drive/Flickr_Data/test_example2.jpg'
new_img3 = 'drive/My Drive/Flickr_Data/test_example3.jpg'


# **Prepare Photo Data**

We will use a pre-trained model to interpret the content of the photos.

There are many models to choose from. In this case, we will use VGG.
Keras provides this pre-trained model directly. Note, the first time you use this model, Keras will download the model weights from the Internet, which are about 500 Megabytes. This may take a few minutes depending on your internet connection.

We could use this model as part of a broader image caption model. The problem is, it is a large model and running each photo through the network every time we want to test a new language model configuration (downstream) is redundant.

Instead, we can pre-compute the “photo features” using the pre-trained model and save them to file. We can then load these features later and feed them into our model as the interpretation of a given photo in the dataset. It is no different to running the photo through the full VGG model; it is just we will have done it once in advance.

This is an optimization that will make training our models faster and consume less memory.

We can load the VGG model in Keras using the VGG class. We will remove the last layer from the loaded model, as this is the model used to predict a classification for a photo. We are not interested in classifying images, but we are interested in the internal representation of the photo right before a classification is made. These are the “features” that the model has extracted from the photo.

Keras also provides tools for reshaping the loaded photo into the preferred size for the model (e.g. 3 channel 224 x 224 pixel image).

**Ex 1:** Define a function named extract_features() that, given a directory name, will load each photo, prepare it for VGG, and collect the predicted features from the VGG model. The image features are a 1-dimensional 4,096 element vector.

This function should returns a dictionary of image identifier to image features.

In [5]:
from os import listdir
from pickle import dump
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.models import Model

In [10]:
# extract features from each photo in the directory
def extract_features(directory):
	# load the model
	model = VGG16()
  # re-structure the model
	#model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
	# summarize
	#print(model.summary())
	# extract features from each photo
	features = dict()
	for name in listdir(directory):
		# load an image from file
		#print(name)
		filename = directory + '/' + name
		#print(filename)
		image = load_img(filename, target_size=(224, 224))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the VGG model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=0)
		# get image id
		image_id = name.split('.')[0]
		# store feature
		features[image_id] = feature
		#print('>%s' % name)
	return features

# extract features from all images
directory = images_path
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('features.pkl', 'wb'))

Extracted Features: 8091


In [11]:
!cp -r features.pkl 'drive/My Drive/Flickr_Data/'

In [6]:
os.listdir('drive/My Drive/Flickr_Data')

['Images',
 'Flickr_TextData',
 'flickr8ktextfiles',
 'test_example1.jpg',
 'test_example2.jpg',
 'test_example3.jpg',
 'features.pkl']

In [7]:
features_file = 'drive/My Drive/Flickr_Data/features.pkl'

We can call this function to prepare the photo data for testing our models, then save the resulting dictionary to a file named ‘features.pkl‘.

Testing

In [13]:
print('Extracted Features: %d' % len(features))  # print No. of features extracted
print(features['123997871_6a9ca987b1'])          # print feature vector of img_id = '123997871_6a9ca987b1'
print(features['141755292_7a0b3364cf'])          # print feature vector of img_id = '141755292_7a0b3364cf'
print(features['141755292_7a0b3364cf'].shape)    # display shape of each feature 

Extracted Features: 8091
[[0.       0.       0.       ... 0.       0.       2.790043]]
[[2.7608128  0.7352772  0.80051225 ... 0.         0.         2.424081  ]]
(1, 4096)


# **Prepare Text Data**

The dataset contains multiple descriptions for each photograph and the text of the descriptions requires some minimal cleaning

1. loading the imagetokens document into memory
2. creating a dictionary from loaded document with imageid's as key's and list of captions as correspoinding value's
3. clean the image descriptions of the dictionary and save to a file
4. create a vocabulary from the cleaned descriptions

**Ex 2:** Define a function to load the file containing all of the image descriptions into memory

In [8]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text


Testing

In [ ]:
filename = captions_path      # assign tokens file 
doc = load_doc(filename)      # load the tokens file into memory
doc = doc.split('\n')         # split the document on new line
doc[0:10]                     # display the first 10 rows of document



['35506150_cbdb630f4f.jpg#0\tA man in a red jacket is sitting on a bench whilst cooking a meal .',
 '35506150_cbdb630f4f.jpg#1\tA man is sitting on a bench , cooking some food .',
 '35506150_cbdb630f4f.jpg#2\tA man sits on a bench .',
 '35506150_cbdb630f4f.jpg#3\tA man wearing a red jacket is sitting on a wooden bench and is cooking something in a small pot .',
 '35506150_cbdb630f4f.jpg#4\ta man wearing a red jacket sitting on a bench next to various camping items',
 '47871819_db55ac4699.jpg#0\tA soccer player in blue is chasing after the player in black and white .',
 '47871819_db55ac4699.jpg#1\tThe girl in the white strip is falling down as the girl in the blue strip challenges for the soccer ball .',
 '47871819_db55ac4699.jpg#2\tThe girls are playing soccer .',
 '47871819_db55ac4699.jpg#3\tTwo women in soccer uniforms playing soccer .',
 '47871819_db55ac4699.jpg#4\tTwo young women on different teams are playing soccer on a field .']

**Ex 3:** define a function load_descriptions() that, given the loaded document text, will return a dictionary of photo identifiers to descriptions. Each photo identifier maps to a list of one or more textual descriptions.

In [9]:
# extract descriptions for images
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:      # skip the lines with words < 2
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# remove filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping


Testing

In [ ]:
text="""101654506_8eb26cfb60.jpg#0	A brown and white dog is running through the snow .
101654506_8eb26cfb60.jpg#1	A dog is running in the snow
101654506_8eb26cfb60.jpg#2	A dog running through snow .
101654506_8eb26cfb60.jpg#3	a white and brown dog is running through a snow covered field .
101654506_8eb26cfb60.jpg#4	The white and brown dog is running over the surface of the snow .

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wooden cabin ."""

descriptions = load_descriptions(text)               # convert the given text into description directory
print("Number of items: " + str(len(descriptions)))  # find no.of items in dictionary
descriptions
descriptions['101654506_8eb26cfb60']                 # display the value of key '101654506_8eb26cfb60' from dictionary

Number of items: 2


['A brown and white dog is running through the snow .',
 'A dog is running in the snow',
 'A dog running through snow .',
 'a white and brown dog is running through a snow covered field .',
 'The white and brown dog is running over the surface of the snow .']

Next, we need to clean the description text. The descriptions are already tokenized and easy to work with.

We will clean the text in the following ways in order to reduce the size of the vocabulary of words we will need to work with:

Convert all words to lowercase.
Remove all punctuation.
Remove all words that are one character or less in length (e.g. ‘a’).
Remove all words with numbers in them.

**Ex 4:** Define the clean_descriptions() function that, given the dictionary of image identifiers to descriptions, steps through each description and cleans the text.

In [10]:
import string

def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)


Testing

In [ ]:
text="""101654506_8eb26cfb60.jpg#0	A brown and white dog is running through the snow .
101654506_8eb26cfb60.jpg#1	A dog is running in the snow
101654506_8eb26cfb60.jpg#2	A dog running through snow .
101654506_8eb26cfb60.jpg#3	a white and brown dog is running through a snow covered field .
101654506_8eb26cfb60.jpg#4	The white and brown dog is running over the surface of the snow .

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wooden cabin ."""


img_descriptions = load_descriptions(text)        # create a description dictionary of given text
print('\n Before Cleaning the Dictionary\n')      
print(img_descriptions['101654506_8eb26cfb60'])   # display the value_list corresponding to key '101654506_8eb26cfb60'
print('\n After Cleaning the Dictionary\n')
clean_descriptions(img_descriptions)              # clean the dictionary created on text
print(img_descriptions['101654506_8eb26cfb60'])   # again display the value_list corresponding to the same key


 Before Cleaning the Dictionary

['A brown and white dog is running through the snow .', 'A dog is running in the snow', 'A dog running through snow .', 'a white and brown dog is running through a snow covered field .', 'The white and brown dog is running over the surface of the snow .']

 After Cleaning the Dictionary

['brown and white dog is running through the snow', 'dog is running in the snow', 'dog running through snow', 'white and brown dog is running through snow covered field', 'the white and brown dog is running over the surface of the snow']


Once cleaned, we can summarize the size of the vocabulary.

Ideally, we want a vocabulary that is both expressive and as small as possible. A smaller vocabulary will result in a smaller model that will train faster.

For reference, we can transform the clean descriptions into a set and print its size to get an idea of the size of our dataset vocabulary.Finally, we can save the dictionary of image identifiers and descriptions to a new file named descriptions.txt, with one image identifier and description per line.

**Ex 5:** Define a function 'to_vocabulary' to transform loaded descriptions. Define another funtion to save these descriptions into a text file

In [11]:
# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# save descriptions to file, one per line
def save_descriptions(descriptions, filename):
	lines = list()
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc)
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

Testing

In [ ]:
text="""101654506_8eb26cfb60.jpg#0	A brown and white dog is running through the snow .
101654506_8eb26cfb60.jpg#1	A dog is running in the snow
101654506_8eb26cfb60.jpg#2	A dog running through snow .
101654506_8eb26cfb60.jpg#3	a white and brown dog is running through a snow covered field .
101654506_8eb26cfb60.jpg#4	The white and brown dog is running over the surface of the snow .

1000268201_693b08cb0e.jpg#0	A child in a pink dress is climbing up a set of stairs in an entry way .
1000268201_693b08cb0e.jpg#1	A girl going into a wooden building .
1000268201_693b08cb0e.jpg#2	A little girl climbing into a wooden playhouse .
1000268201_693b08cb0e.jpg#3	A little girl climbing the stairs to her playhouse .
1000268201_693b08cb0e.jpg#4	A little girl in a pink dress going into a wooden cabin ."""

descriptions = load_descriptions(text)                 # parse text into dictionary
print('Items in Dictionary: %d ' % len(descriptions))  # print length of dictionary

clean_descriptions(descriptions)                     # clean dictionary

vocabulary = to_vocabulary(descriptions)             # summarize the vocabulary of dictionary
print('Vocabulary is : ', vocabulary)               

# print vocabulary
print('Vocabulary Size: %d' % len(vocabulary))       # display size of vocabulary

save_descriptions(descriptions, 'descriptions.txt')  # save vocabulary to file 'descriptions.txt'
filename = 'descriptions.txt'                        
doc = load_doc(filename).split('\n')                 # load the saved file and split on new line
print('descriptions.txt :')                          # display the loaded text file
doc

Items in Dictionary: 2 
Vocabulary is :  {'white', 'building', 'girl', 'dog', 'is', 'wooden', 'set', 'snow', 'her', 'playhouse', 'up', 'pink', 'of', 'an', 'entry', 'in', 'through', 'stairs', 'to', 'covered', 'dress', 'and', 'child', 'way', 'the', 'brown', 'climbing', 'cabin', 'going', 'field', 'over', 'into', 'running', 'surface', 'little'}
Vocabulary Size: 35
descriptions.txt :


['101654506_8eb26cfb60 brown and white dog is running through the snow',
 '101654506_8eb26cfb60 dog is running in the snow',
 '101654506_8eb26cfb60 dog running through snow',
 '101654506_8eb26cfb60 white and brown dog is running through snow covered field',
 '101654506_8eb26cfb60 the white and brown dog is running over the surface of the snow',
 '1000268201_693b08cb0e child in pink dress is climbing up set of stairs in an entry way',
 '1000268201_693b08cb0e girl going into wooden building',
 '1000268201_693b08cb0e little girl climbing into wooden playhouse',
 '1000268201_693b08cb0e little girl climbing the stairs to her playhouse',
 '1000268201_693b08cb0e little girl in pink dress going into wooden cabin']

**Ex 5A:** Write the code to prepare and store the descriptions of token file

In [12]:
filename = captions_path
# load the file as text 
doc = load_doc(filename)
# parse text into descriptions
token_descriptions = load_descriptions(doc)
# clean descriptions
clean_descriptions(token_descriptions)
# summarize vocabulary
vocabulary = to_vocabulary(token_descriptions)
# save to file
save_descriptions(token_descriptions, 'descriptions.txt')

Testing:

In [19]:
print('Items in description dictionary: %d ' % len(token_descriptions))      # display length of token_descriptions
print('Token data Vocabulary Size: %d' % len(vocabulary))                    # display voabulary size
print(token_descriptions['35506150_cbdb630f4f'])                             # display the token_descriptions of key value: '35506150_cbdb630f4f'

Items in description dictionary: 8092 
Token data Vocabulary Size: 8763
['man in red jacket is sitting on bench whilst cooking meal', 'man is sitting on bench cooking some food', 'man sits on bench', 'man wearing red jacket is sitting on wooden bench and is cooking something in small pot', 'man wearing red jacket sitting on bench next to various camping items']


# **Develop Deep Learning Model**

In this section, we will define the deep learning model and fit it on the training dataset.

This section is divided into the following parts:

1. Loading Data.
2. Defining the Model.
3. Fitting the Model.

Loading Data

First, we must load the prepared photo and text data so that we can use it to fit the model.

We are going to train the data on all of the photos and captions in the training dataset. While training, we are going to monitor the performance of the model on the development dataset and use that performance to decide when to save models to file.

The train and development dataset have been predefined in the Flickr_8k.trainImages.txt and Flickr_8k.devImages.txt files respectively, that both contain lists of photo file names. From these file names, we can extract the photo identifiers and use these identifiers to filter photos and descriptions for each set.

**Ex 6:** define a function load_set() to load a pre-defined set of identifiers given the train or development sets filename.

In [13]:
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

Testing

In [24]:
file = val_path          # choose validation set of images
val = load_set(file)     # create a list of image id's on called file
print(val)               # print the list
print(len(val))          # display the length of list

{'2053733930_e245615ad4', '454709143_9c513f095c', '3247693965_845b3b4349', '3583321426_f373c52161', '3693961165_9d6c333d5b', '3523819210_99782628fc', '1579798212_d30844b4c5', '1420060118_aed262d606', '3341961913_9a9b362f15', '3432634159_1eb9a73306', '2121140070_a09644550b', '2300229745_af7a20c959', '3480052428_c034b98a08', '3331009729_d3b14738e6', '3047751696_78c2efe5e6', '262446581_06ef7d9445', '3265864834_e0229020dd', '3488087117_2719647989', '3516285214_59823b341e', '294098577_c10f32bcfa', '2578289278_01516d23a0', '3420338549_bd78d35243', '3330935489_9cb67ca36b', '2097489021_ca1b9f5c3b', '3421547427_53411b6278', '2293149170_38fb2257ea', '140526327_3cb984de09', '203114209_e4cd71a6b7', '134724228_30408cd77f', '3601978895_9fec23ce0c', '656260720_a7db4ce48b', '3436259762_43709321ff', '247706586_7e25c7adf8', '1138784872_69ade3f2ab', '2431723485_bc6b8e6418', '3438981089_2ef1a6353c', '191003283_992257f835', '2493974889_50ae29f1e1', '2806632713_edd6f6c893', '3088922727_4bdf2ccc94', '3393446

Now, we can load the photos and descriptions using the pre-defined set of train or development identifiers.

Below is the function load_clean_descriptions() that loads the cleaned text descriptions from ‘descriptions.txt‘ for a given set of identifiers and returns a dictionary of identifiers to lists of text descriptions.

The model we will develop will generate a caption given a photo, and the caption will be generated one word at a time. The sequence of previously generated words will be provided as input. Therefore, we will need a ‘first word’ to kick-off the generation process and a ‘last word‘ to signal the end of the caption.

We will use the strings ‘startseq‘ and ‘endseq‘ for this purpose. These tokens are added to the loaded descriptions as they are loaded. It is important to do this now before we encode the text so that the tokens are also encoded correctly.

**Ex 7:** Define a function load_clean_descriptions() that loads the cleaned text descriptions from ‘descriptions.txt‘ for a given set of identifiers and returns a dictionary of identifiers to lists of text descriptions.


In [14]:
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions

Testing

In [ ]:
                      
file = val_path          # path of train set of images
val_data = load_set(file)     # create a list of image id's on called file
data_descriptions = load_clean_descriptions('descriptions.txt', val)
print(data_descriptions['346253487_687150ab04'])  # display cleaned descriptions of the keys "346253487_687150ab04" & "356143774_ef3e93eede"
print(data_descriptions['356143774_ef3e93eede'])  


Next, we can load the photo features for a given dataset.

This is not very efficient; nevertheless, this will get us up and running quickly.

**Ex 8:** Define a function named load_photo_features() that loads the entire set of photo descriptions, then returns the subset of interest for a given set of photo identifiers

In [15]:
# load photo features
from pickle import load
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

Testing

In [16]:
# load training dataset 
filename = train_path
train = load_set(filename)
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print(train_descriptions)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
#train_features = load_photo_features('features.pkl', train)
train_features = load_photo_features(features_file, train)
print('Photos: train=%d' % len(train_features))


Dataset: 6000
{'1000268201_693b08cb0e': ['startseq child in pink dress is climbing up set of stairs in an entry way endseq', 'startseq girl going into wooden building endseq', 'startseq little girl climbing into wooden playhouse endseq', 'startseq little girl climbing the stairs to her playhouse endseq', 'startseq little girl in pink dress going into wooden cabin endseq'], '1001773457_577c3a7d70': ['startseq black dog and spotted dog are fighting endseq', 'startseq black dog and tricolored dog playing with each other on the road endseq', 'startseq black dog and white dog with brown spots are staring at each other in the street endseq', 'startseq two dogs of different breeds looking at each other on the road endseq', 'startseq two dogs on pavement moving toward each other endseq'], '1002674143_1b742ab4b8': ['startseq little girl covered in paint sits in front of painted rainbow with her hands in bowl endseq', 'startseq little girl is sitting in front of large painted rainbow endseq', 's

The description text will need to be encoded to numbers before it can be presented to the model as in input or compared to the model’s predictions.

The first step in encoding the data is to create a consistent mapping from words to unique integer values. Keras provides the Tokenizer class that can learn this mapping from the loaded description data.

**Ex 9:** Define the functions 'to_lines()' to convert the dictionary of descriptions into a list of strings and the 'create_tokenizer()' function that will fit a Tokenizer given the loaded photo description text.. 

In [16]:
# convert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer


In [18]:
# prepare tokenizer on train descriptions 
tokenizer = create_tokenizer(train_descriptions)
print(tokenizer.index_word)
vocab_size = len(tokenizer.word_index) + 1
print('Train data Vocabulary Size: %d' % vocab_size)

{1: 'startseq', 2: 'endseq', 3: 'in', 4: 'the', 5: 'on', 6: 'is', 7: 'and', 8: 'dog', 9: 'with', 10: 'man', 11: 'of', 12: 'two', 13: 'white', 14: 'black', 15: 'boy', 16: 'are', 17: 'woman', 18: 'girl', 19: 'to', 20: 'wearing', 21: 'at', 22: 'people', 23: 'water', 24: 'brown', 25: 'young', 26: 'red', 27: 'an', 28: 'his', 29: 'blue', 30: 'dogs', 31: 'running', 32: 'through', 33: 'playing', 34: 'while', 35: 'down', 36: 'shirt', 37: 'ball', 38: 'standing', 39: 'little', 40: 'grass', 41: 'snow', 42: 'child', 43: 'person', 44: 'jumping', 45: 'over', 46: 'three', 47: 'sitting', 48: 'front', 49: 'field', 50: 'holding', 51: 'small', 52: 'yellow', 53: 'green', 54: 'group', 55: 'up', 56: 'by', 57: 'large', 58: 'one', 59: 'walking', 60: 'her', 61: 'men', 62: 'children', 63: 'air', 64: 'into', 65: 'near', 66: 'mouth', 67: 'beach', 68: 'jumps', 69: 'runs', 70: 'another', 71: 'for', 72: 'street', 73: 'from', 74: 'its', 75: 'riding', 76: 'stands', 77: 'bike', 78: 'girls', 79: 'as', 80: 'outside', 81: 

We can now encode the text.

Each description will be split into words. The model will be provided one word and the photo and generate the next word. Then the first two words of the description will be provided to the model as input with the image to generate the next word. This is how the model will be trained.

For example, the input sequence “little girl running in field” would be split into 6 input-output pairs to train the model:

In [ ]:
# not runnable code


X1,		X2 (text sequence), 						             y (word)
photo	startseq, 									                 little
photo	startseq, little,							               girl
photo	startseq, little, girl, 					           running
photo	startseq, little, girl, running, 			       in
photo	startseq, little, girl, running, in, 		     field
photo	startseq, little, girl, running, in, field,  endseq

Later, when the model is used to generate descriptions, the generated words will be concatenated and recursively provided as input to generate a caption for an image.

The function below named create_sequences(), given the tokenizer, a maximum sequence length, and the dictionary of all descriptions and photos, will transform the data into input-output pairs of data for training the model. There are two input arrays to the model: one for photo features and one for the encoded text. There is one output for the model which is the encoded next word in the text sequence.

The input text is encoded as integers, which will be fed to a word embedding layer. The photo features will be fed directly to another part of the model. The model will output a prediction, which will be a probability distribution over all words in the vocabulary.

The output data will therefore be a one-hot encoded version of each word, representing an idealized probability distribution with 0 values at all word positions except the actual word position, which has a value of 1.

**Ex 10:** Define a function named create_sequences() in such a way that, given the tokenizer, a maximum sequence length, and the dictionary of all descriptions and photos, will transform the data into input-output pairs of data for training the model.

Define another function named max_length() to calculate the maximum number of words in the longest description.

In [17]:
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each image identifier
	for key, desc_list in descriptions.items():
		# walk through each description for the image
		for desc in desc_list:
			# encode the sequence
			seq = tokenizer.texts_to_sequences([desc])[0]
			# split one sequence into multiple X,y pairs
			for i in range(1, len(seq)):
				# split into input and output pair
				in_seq, out_seq = seq[:i], seq[i]
				# pad input sequence
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				# encode output sequence
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
				# store
				X1.append(photos[key][0])
				X2.append(in_seq)
				y.append(out_seq)
	return array(X1), array(X2), array(y)
 
 # calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

Testing:

In [ ]:
test={'367925122_335ed279a8', '386160015_d4b31df68e'}
print("Length of test set: ",len(test))
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print(test_descriptions)
max_length = max_length(test_descriptions)
# photo features
test_features = load_photo_features('features.pkl', test)
#print(test_features['367925122_335ed279a8'])
# prepare sequences
x1test, x2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features, vocab_size)
print('\n',x1test.shape)
print(x1test)
print(x2test.shape)
print(x2test)
print(ytest.shape)
print(ytest)


Length of test set:  2
{'367925122_335ed279a8': ['startseq blonde toddler in light green shirt sitting in green chair surrounded by other green chairs endseq', 'startseq cute toddler girl waits in her stroller endseq', 'startseq little blonde girl is sitting on green chair endseq', 'startseq little girl is turned around in her seat at an event endseq', 'startseq young girl with blond hair holding onto rail endseq'], '386160015_d4b31df68e': ['startseq brown dog playing play wrestling with black dog on snow covered land endseq', 'startseq the two dogs are fighting outside in the snow endseq', 'startseq two brown dogs playing in snow endseq', 'startseq two dogs are wrestling in snow covered field endseq', 'startseq two dogs playing on snow covered ground endseq']}

 (69, 4096)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.      


We now have enough to load the data for the training and development datasets and transform the loaded data into input-output pairs for fitting a deep learning model.

**Ex 11:** Write the code to prepare sequences of training and validation image data sets

In [ ]:
# train dataset

# load training dataset (6K)
filename = train_path
train = load_set(filename)
print('Length of Train Dataset:', len(train))
# descriptions
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions of Train set:', len(train_descriptions))
# photo features
#train_features = load_photo_features('features.pkl', train)
train_features = load_photo_features(features_file, train)
print('Length of Train features:',len(train_features))
# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))
vocab_size = len(tokenizer.word_index) + 1
print('Train data Vocabulary Size: ', vocab_size)
# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Max Description Length: ', max_length)
# prepare sequences
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)

# dev dataset
# load validation set
filename = val_path
val = load_set(filename)
print('Length of Val Dataset:', len(val))
# descriptions
val_descriptions = load_clean_descriptions('descriptions.txt', val)
print('Descriptions of Val set:', len(val_descriptions))
# photo features
#val_features = load_photo_features('features.pkl', val)
val_features = load_photo_features(features_file, val)
print('Length of Val features:', len(val_features))
# prepare sequences
X1val, X2val, yval = create_sequences(tokenizer, max_length, val_descriptions, val_features, vocab_size)



Length of Train Dataset: 6000
Descriptions of Train set: 6000
Length of Train features: 6000
Train data Vocabulary Size:  7579
Max Description Length:  34


Testing: 

In [ ]:
print('Train data details: \n')
print(X1train.shape)          # display the shape of x1train sequences
print(X2train.shape)          # display the shape of x2train sequences
print(ytrain.shape)           # display the shape of ytrain sequences
print(ytrain)                 # display ytrain data

print('\nValidation data details:\n')
print(X1val.shape)           # similary display for val data sequences 
print(X2val.shape)
print(yval.shape)
print(yval)

Train data details: 

(1061, 4096)
(1061, 26)
(1061, 323)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]

Validation data details:

(319, 4096)
(319, 26)
(319, 323)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]



# **Defining & Fitting the Model**

Problem Statement

We will describe the model in three parts:

**Photo Feature Extractor:** This is a 16-layer VGG model pre-trained on the ImageNet dataset. We have pre-processed the photos with the VGG model (without the output layer) and will use the extracted features predicted by this model as input.

**Sequence Processor**: This is a word embedding layer for handling the text input, followed by a Long Short-Term Memory (LSTM) recurrent neural network layer.

**Decoder** (for lack of a better name). Both the feature extractor and sequence processor output a fixed-length vector. These are merged together and processed by a Dense layer to make a final prediction.
The Photo Feature Extractor model expects input photo features to be a vector of 4,096 elements. These are processed by a Dense layer to produce a 256 element representation of the photo.

The Sequence Processor model expects input sequences with a pre-defined length (26 words) which are fed into an Embedding layer that uses a mask to ignore padded values. This is followed by an LSTM layer with 256 memory units.

Both the input models produce a 256 element vector. Further, both input models use regularization in the form of 50% dropout. This is to reduce overfitting the training dataset, as this model configuration learns very fast.

The Decoder model merges the vectors from both input models using an addition operation. This is then fed to a Dense 256 neuron layer and then to a final output Dense layer that makes a softmax prediction over the entire output vocabulary for the next word in the sequence.

**Ex 12:** Define a function named define_model() with the information given in the problem statement and returns the model ready to fit.

In [ ]:
import tensorflow as tf
from keras.layers import Dense, Embedding, Dropout, LSTM, Input
from keras.models import Model

In [ ]:
# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(4096,))              # Input layer of shape (4096,)
	fe1 = Dropout(0.5)(inputs1)                 # dropout layer with 50% rate 
	fe2 = Dense(256, activation='relu')(fe1)    # dense layer with 256 neurons, relu activation 
	# sequence model
	inputs2 = Input(shape=(max_length,))        # input layer of shape (max_length,)
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2) # embedding layer with vocab_size, 256 units, and set mask_zero to 'true'
	se2 = Dropout(0.5)(se1)                     # dropout layer with 50% rate
	se3 = LSTM(256)(se2)                        # LSTM with 256 units
	# decoder model
	decoder1 = add([fe2, se3])                       
	decoder2 = Dense(256, activation='relu')(decoder1)          # dense layer of 256 units, relu activation
	outputs = Dense(vocab_size, activation='softmax')(decoder2) # another dense layer of vocab_size units, softmax activation
	# tie it together [image, seq] [word]          
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)   
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	return model

# Fitting the model with generated training and validation data sequences 
import tensorflow as tf
from tensorflow.keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

model = define_model(vocab_size, max_length)    # call the model with given parameters
print(model.summary())                          # display the model summary
# define checkpoint callback
#filepath = 'model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
filepath = 'Mymodel.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
# fit model
model.fit([X1train, X2train], ytrain, epochs=20, verbose=2, callbacks=[checkpoint], validation_data=([X1val, X2val], yval))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 26)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 4096)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 26, 256)      133120      input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 4096)         0           input_2[0][0]                    
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2085 samples, validate on 712 samples
Epoch 1/20
 - 14s - loss: 5.5703 - val_loss: 5.1876

Epoch 00001: val_loss improved from inf to 5.18760, saving model to Mymodel.h5
Epoch 2/20
 - 13s - loss: 4.5507 - val_loss: 5.1408

Epoch 00002: val_loss improved from 5.18760 to 5.14084, saving model to Mymodel.h5
Epoch 3/20
 - 13s - loss: 3.9702 - val_loss: 5.1323

Epoch 00003: val_loss improved from 5.14084 to 5.13229, saving model to Mymodel.h5
Epoch 4/20
 - 13s - loss: 3.6031 - val_loss: 5.1499

Epoch 00004: val_loss did not improve from 5.13229
Epoch 5/20
 - 13s - loss: 3.3790 - val_loss: 5.0243

Epoch 00005: val_loss improved from 5.13229 to 5.02435, saving model to Mymodel.h5
Epoch 6/20
 - 13s - loss: 3.2304 - val_loss: 5.0026

Epoch 00006: val_loss improved from 5.02435 to 5.00255, saving model to Mymodel.h5
Epoch 7/20
 - 13s - loss: 3.0886 - val_loss: 4.9562

Epoch 00007: val_loss improved from 5.00255 to 4.95618, saving model to Mymodel.h5
Epoch 8/20
 - 13s - loss: 2.9421 - va

# **Evaluate Model**

Once the model is fit, we can evaluate the skill of its predictions on the holdout test dataset.

We will evaluate a model by generating descriptions for all photos in the test dataset and evaluating those predictions with a standard cost function.

First, we need to be able to generate a description for a photo using a trained model.

This involves passing in the start description token ‘startseq‘, generating one word, then calling the model recursively with generated words as input until the end of sequence token is reached ‘endseq‘ or the maximum description length is reached.

The function below named generate_desc() implements this behavior and generates a textual description given a trained model, and a given prepared photo as input. It calls the function word_for_id() in order to map an integer prediction back to a word.

**Ex 13:** Define a function named generate_desc() implements the above said behavior and generates a textual description given a trained model, and a given prepared photo as input. It calls the function word_for_id() in order to map an integer prediction back to a word.

In [ ]:
from numpy import argmax

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process by assigning 'startseq'
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text



Testing

In [ ]:
photo = val_features['347543966_b2053ae78c']  
# step over the whole set
for key, desc_list in val_descriptions.items():
  # generate description
  pred = generate_desc(model, tokenizer, photo, max_length)
  print(pred)
  break

startseq two girl are playing on the the the one pose endseq


We will generate predictions for all photos in the test dataset and in the train dataset.

The function below named evaluate_model() will evaluate a trained model against a given dataset of photo descriptions and photo features. The actual and predicted descriptions are collected and evaluated collectively using the corpus BLEU score that summarizes how close the generated text is to the expected text.

**Ex 14:** define a function evaluate_model() that will evaluate a trained model against a given dataset of photo descriptions and photo features. The actual and predicted descriptions are collected and evaluated collectively using the corpus BLEU score that summarizes how close the generated text is to the expected text.

In [ ]:
# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

Testing:

In [ ]:
# load test set
filename = test_path
test = load_set(filename)
print('Dataset: %d' % len(test))
# descriptions
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=%d' % len(test_descriptions))
# photo features
test_features = load_photo_features('features.pkl', test)
print('Photos: test=%d' % len(test_features))

# load the model
filename = 'Mymodel.h5'
model = load_model(filename)
# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 20
Descriptions: test=20
Photos: test=20
BLEU-1: 0.397314
BLEU-2: 0.105977
BLEU-3: 0.239384
BLEU-4: 0.293472


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


New predictions

**Ex 15:** Write the code to predict caption for given a new imagefile  

In [ ]:

# extract features from each photo in the directory
def extract_features(filename):
	# load the model
	model = VGG16()
	# re-structure the model
	model.layers.pop()
	model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
	# load the photo
	image = load_img(filename, target_size=(224, 224))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the VGG model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=0)
	return feature

# load the model
model = load_model('Mymodel.h5')
# load and prepare the photograph
photo = extract_features(new_img2)
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

startseq bird bird stands standing the water endseq


In [ ]:
model = load_model('Mymodel.h5')
# load and prepare the photograph
photo = extract_features(new_img1)
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

startseq girl in in is is endseq


In [ ]:
model = load_model('Mymodel.h5')
# load and prepare the photograph
photo = extract_features(new_img3)
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

startseq man and and woman serve endseq


# ************ THE END ************